In [83]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import xml.etree.ElementTree as et

import xmltodict


import json

from IPython.display import display

In [84]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [85]:
query_in = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="all")
query_out = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="out")
query_item = pd.read_excel(r"C:\Users\Samsung\Desktop\all\Project\23_VRPTW\Data\Korea\Query_Book.xlsx",sheet_name="items")
query_in.head()

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
0,serviceKey,인증키,100,1,인증키,공공데이터포털에서 발급받은 인증키,이용자
1,numOfRows,한 페이지 결과 수,4,1,2,한 페이지 결과 수,이용자
2,pageNo,페이지 번호,4,1,1,페이지 번호,이용자
3,dataType,응답자료형식,4,0,xml,요청자료형식(xml/json),이용자
4,stdrYm,기준년월,6,1,201702,요청 조회 날짜 (YYYYMM),이용자


In [86]:
query_out.head()

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
0,numOfRows,한 페이지 결과 수,4,1,2,한 페이지당 표출,이용자
1,pageNo,페이지 번호,4,1,1,페이지 수,이용자
2,totalCount,데이터 총 개수,10,1,7932,데이터 총 개수,이용자
3,resultCode,응답메시지 코드,2,1,0,응답 메시지코드,이용자
4,resultMsg,응답메시지 내용,100,1,NORMAL SERVICE,응답 메시지 설명,이용자


In [87]:
def query_df_out(query_df,query,version=None):
    a = query_df[query_df["구분"]==query]
    if version =="all":
        b = a
    elif version == 1:
        b = a[query_df["항목구분"]==0]
    else:
        b = a[query_df["항목구분"]==1]
    return b

## Making dictionary

In [88]:
qin_1 = query_in[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)
qin_1.head()

,항목명(영문),항목명(국문)
0,serviceKey,인증키
1,numOfRows,한 페이지 결과 수
2,pageNo,페이지 번호
3,dataType,응답자료형식
4,stdrYm,기준년월


In [89]:
qout_2 = query_out[["항목명(영문)","항목명(국문)"]].drop_duplicates(subset=["항목명(영문)"]).reset_index(drop=True)
qout_2.head()

,항목명(영문),항목명(국문)
0,numOfRows,한 페이지 결과 수
1,pageNo,페이지 번호
2,totalCount,데이터 총 개수
3,resultCode,응답메시지 코드
4,resultMsg,응답메시지 내용


In [90]:
query_item.head()

,코드,명,구분
0,1,중앙,areaCd
1,2,서울,areaCd
2,3,부산,areaCd
3,4,대구,areaCd
4,5,인천,areaCd


In [91]:
cate_ = {}
for row,col in qin_1.iterrows():
    key_ = col["항목명(영문)"]
    value_  =col["항목명(국문)"]
    cate_[key_] = value_

for row,col in qout_2.iterrows():
    key_ = col["항목명(영문)"]
    value_  =col["항목명(국문)"]
    cate_[key_] = value_


In [92]:
cate_

{'serviceKey': '인증키',
 'numOfRows': '한 페이지 결과 수',
 'pageNo': '페이지 번호',
 'dataType': '데이터 타입',
 'stdrYm': '기준년월',
 'areaCd': '지역코드',
 'unitySignguCd': '통합시군구코드',
 'spctrCd': '지원센터코드',
 'userSeccd': '이용자구분코드',
 'userClscd': '이용자분류코드',
 'happyTrgterYn': '행복e음대상자여부',
 'fcltySeNm': '시설유형명',
 'fcltyGrpClscd': '시설단체분류코드',
 'cntrbtrSeccd': '기부자구분코드',
 'cntrBplcKndcd': '기부사업장종류코드',
 'cntrbtrNm': '기부자명',
 'rceptSpctrCd': '접수지원센터코드',
 'foodYn': '식품여부',
 'cnttgLclasCd': '기부물품대분류코드',
 'cnttgMlsfcCd': '기부물품중분류코드',
 'distbTmlmtUseYn': '유통기한사용여부',
 'injryGoodsIntrcpYn': '유해상품차단여부',
 'spctrUncd': '지원센터단위코드',
 'spctrSecd': '지원센터구분코드',
 'spctrStscd': '지원센터상태코드',
 'operMbyLclasCd': '운영주체대분류코드',
 'operMbySclasCd': '운영주체소분류코드',
 'sttemntSeccd': '신고구분코드',
 'registDe': '등록일자',
 'preferCnttgClscd': '선호물품분류코드',
 'totalCount': '데이터 총 개수',
 'resultCode': '응답메시지 코드',
 'resultMsg': '응답메시지 내용',
 'useAmt': '이용금액',
 'useCo': '이용건수',
 'userCo': '이용자수',
 'fcltyGrpCo': '시설단체수',
 '\\cntrAmt': '기부금액',
 'cntrCo': '기부건수',
 '

In [94]:
sample_ = "http://apis.data.go.kr/B460014/foodBankInfoService2/getPreferInfo?serviceKey=인증키(URL Encode) &numOfRows=2&pageNo=1"
url_split(sample_)[1]

{'serviceKey': '인증키(URL Encode) ', 'numOfRows': '2', 'pageNo': '1'}

In [95]:
query_1 = {
    "getUserInfo": "이용자", "getFcltyGrpInfo" : "이용시설단체", "getCntrbtrInfo":"기부자",
    "getRceptStat": "접수현황", "getProvdStat": "제공현황",
    "getCnttgInfo":"물품현황","getSpctrInfo":"센터통계", "getPreferInfo":"선호물품"
}

In [96]:
url_main = "http://apis.data.go.kr/B460014/foodBankInfoService2"

In [97]:
encode = "%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D"
decode = "/xc7f1IdclYG5tj6DIg1cU51nehX/zyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA=="

# 물품현황

In [98]:
url_test = url_main + "/" + "getCnttgInfo" + "?serviceKey=" + encode
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getCnttgInfo?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D'

In [100]:
paras_ = {
    "numOfRows": 0,
    "pageNo": 0,
    "stdrYm": 0
}

In [101]:
paras_["numOfRows"] = 100
paras_["pageNo"] = 1
paras_["stdrYm"] = 202010


In [102]:

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [103]:
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getCnttgInfo?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D&numOfRows=100&pageNo=1&stdrYm=202010'

In [104]:
html = requests.get(url_test).text
#soup = BeautifulSoup(html,"html.parser")
#soup

In [105]:
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]

In [106]:
df_out = pd.DataFrame(dictionay)
df_out

,stdrYm,distbTmlmtUseYn,injryGoodsIntrcpYn,foodYn,cnttgLclasCd,cnttgMlsfcCd,cnttgQy,acntbkAmt
0,202010,N,Y,Y,01000000,01160000,0,0
1,202010,N,N,Y,02000000,02830000,0,0
2,202010,N,N,Y,01000000,01020000,324,239600
3,202010,N,N,Y,01000000,01170000,2577,10341740.45
4,202010,N,N,Y,02000000,02990000,0,0
...,...,...,...,...,...,...,...,...
95,202010,Y,N,Y,02000000,02820000,278,941640
96,202010,Y,N,Y,02000000,02830000,0,0
97,202010,Y,N,Y,02000000,02880000,0,0
98,202010,Y,N,Y,02000000,02990000,35,260000


In [107]:

for col_ in df_out.columns:
    target_ = query_item[query_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        data_ = df_out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        data_2 = [dic_tar[i] for i in data_.values]
        df_out[col_] = data_2
    else:
        pass

In [108]:
df_out = df_out.rename(columns={col:cate_[col] for col in df_out.columns})
df_out

,기준년월,유통기한사용여부,유해상품차단여부,식품여부,기부물품대분류코드,기부물품중분류코드,기부물품수량,장부금액
0,202010,N,Y,Y,가공식품,과자류,0,0
1,202010,N,N,Y,신선식품,수산물(냉동),0,0
2,202010,N,N,Y,가공식품,유제품,324,239600
3,202010,N,N,Y,가공식품,음료류,2577,10341740.45
4,202010,N,N,Y,신선식품,기타신선식품,0,0
...,...,...,...,...,...,...,...,...
95,202010,Y,N,Y,신선식품,축산물(냉동),278,941640
96,202010,Y,N,Y,신선식품,수산물(냉동),0,0
97,202010,Y,N,Y,신선식품,신선식품세트류,0,0
98,202010,Y,N,Y,신선식품,기타신선식품,35,260000


In [109]:
url_test = url_main + "/" + "getSpctrInfo" + "?serviceKey=" + encode
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getSpctrInfo?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D'

In [110]:
query_in["구분"].value_counts()

센터        15
이용자       11
기부자       11
접수현황      11
제공현황      11
이용시설단체    10
물품현황      10
선호물품       8
Name: 구분, dtype: int64

In [111]:
query_df_out(query_in,"물품현황","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
54,serviceKey,인증키,100,1,인증키,공공데이터포털에서 발급받은 인증키,물품현황
55,numOfRows,한 페이지 결과 수,4,1,2,한 페이지 결과 수,물품현황
56,pageNo,페이지 번호,4,1,1,페이지 번호,물품현황
57,dataType,응답자료형식,4,0,xml,요청자료형식 (xml/json),물품현황
58,stdrYm,기준년월,6,1,201702,요청 조회 날짜 (YYYYMM),물품현황
59,distbTmlmtUseYn,유통기한사용여부,1,0,N,요청 유통기한사용여부 (Y/N),물품현황
60,injryGoodsIntrcpYn,유해상품차단여부,1,0,Y,요청 유해상품차단여부 (Y/N),물품현황
61,foodYn,식품여부,1,0,Y,요청 식품여부 (Y/N),물품현황
62,cnttgLclasCd,기부물품대분류코드,10,0,02000000(신선식품),요청 기부물품대분류 코드,물품현황
63,cnttgMlsfcCd,기부물품중분류코드,10,0,02010000(농산물),요청 기부물품중분류 코드,물품현황


In [113]:
query_df_out(query_out,"물품","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
80,numOfRows,한 페이지 결과 수,4,1,2,한 페이지당 표출,물품
81,pageNo,페이지 번호,4,1,1,페이지 수,물품
82,totalCount,데이터 총 개수,10,1,105,데이터 총 개수,물품
83,resultCode,응답메시지 코드,2,1,0,응답 메시지코드,물품
84,resultMsg,응답메시지 내용,100,1,NORMAL SERVICE,응답 메시지 설명,물품
85,dataType,데이터 타입,4,1,XML,응답,물품
86,stdrYm,기준년월,6,1,201702,응답 조회 날짜 (YYYYMM),물품
87,distbTmlmtUseYn,유통기한사용여부,1,1,N,응답 유통기한사용여부 (Y/N),물품
88,injryGoodsIntrcpYn,유해상품차단여부,1,1,Y,응답 유해상품차단여부 (Y/N),물품
89,foodYn,식품여부,1,1,Y,응답 식품여부 (Y/N),물품


In [76]:
paras_["numOfRows"] = 100
paras_["pageNo"] = 1
paras_["stdrYm"] = 201001

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [77]:
html = requests.get(url_test).text

In [78]:
html

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE</resultMsg></header><body><dataType>XML</dataType><items /><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>\r\n'

In [72]:
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
df_out = pd.DataFrame(dictionay)
df_out

TypeError: 'NoneType' object is not subscriptable

In [55]:
result

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE'},
  'body': {'dataType': 'XML',
   'items': None,
   'numOfRows': '100',
   'pageNo': '1',
   'totalCount': '0'}}}

##  접수현황

In [117]:
url_test = url_main + "/" + "getRceptStat" + "?serviceKey=" + encode
url_test

'http://apis.data.go.kr/B460014/foodBankInfoService2/getRceptStat?serviceKey=%2Fxc7f1IdclYG5tj6DIg1cU51nehX%2FzyMWvw8PpOvhKJqmfs8VwS2fnyJi03lk0J1UC73TJxpw6VBrT4Dk6rwkA%3D%3D'

In [118]:
query_df_out(query_in,"접수현황","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
32,serviceKey,인증키,100,1,인증키,공공데이터포털에서 발급받은 인증키,접수현황
33,numOfRows,한 페이지 결과 수,4,1,2,한 페이지 결과 수,접수현황
34,pageNo,페이지 번호,4,1,1,페이지 번호,접수현황
35,dataType,응답자료형식,4,0,xml,요청자료형식(xml/json),접수현황
36,stdrYm,기준년월,6,1,201702,요청 조회 날짜 (YYYYMM),접수현황
37,areaCd,지역코드,2,0,01(서울),요청 지역코드 코드정보별첨 – 지역코드 정보 참조,접수현황
38,unitySignguCd,통합시군구코드,8,0,01000000(마포구),요청 통합시군구코드 코드정보별첨 – 통합시군구코드 정보 참조,접수현황
39,rceptSpctrCd,접수지원센터코드,6,0,S00101(마포푸드마켓3호점),요청 지원센터 코드 코드정보별첨 – 센터코드 정보 참조,접수현황
40,foodYn,식품여부,1,0,N,요청 식품여부코드 (Y/N),접수현황
41,cnttgLclasCd,기부물품대분류코드,10,0,03000000(가공식품),요청 기부물품대분류코드 코드정보별첨 – 기부물품대분류코드 정보 참조,접수현황


In [119]:
query_df_out(query_out,"접수현황","all")

,항목명(영문),항목명(국문),항목크기,항목구분,샘플데이터,항목설명,구분
47,numOfRows,한 페이지 결과 수,4,1,2,한 페이지당 표출,접수현황
48,pageNo,페이지 번호,4,1,1,페이지 수,접수현황
49,totalCount,데이터 총 개수,10,1,2751,데이터 총 개수,접수현황
50,resultCode,응답메시지 코드,2,1,0,응답 메시지코드,접수현황
51,resultMsg,응답메시지 내용,100,1,NORMAL SERVICE,응답 메시지 설명,접수현황
52,dataType,데이터 타입,4,1,XML,응답,접수현황
53,stdrYm,기준년월,6,1,201702,응답 조회 날짜 (YYYYMM),접수현황
54,areaCd,지역코드,2,1,01(중앙),응답 지역코드 코드정보별첨 – 지역코드 정보 참조,접수현황
55,unitySignguCd,통합시군구코드,8,1,01000000(중앙),응답 통합시군구코드 코드정보별첨 – 통합시군구코드 정보 참조,접수현황
56,rceptSpctrCd,접수지원센터코드,6,1,S00101(전국푸드뱅크),응답 지원센터코드 코드정보별첨 – 센터코드 정보 참조,접수현황


In [132]:
paras_["numOfRows"] = 2
paras_["pageNo"] = 1
paras_["stdrYm"] = 201702

for key,value in paras_.items():
    url_test += "&" + key + "=" + str(value)

In [138]:
test_2 = "http://apis.data.go.kr/B460014/foodBankInfoService2/getRceptStat?serviceKey={}&stdrYm=201702&numOfRows=10&pageNo=1".format(encode)

In [139]:
html = requests.get(test_2).text
html

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><stdrYm>201702</stdrYm><areaCd>01</areaCd><unitySignguCd>01000000</unitySignguCd><rceptSpctrCd>S00101</rceptSpctrCd><foodYn>N</foodYn><cnttgLclasCd>03000000</cnttgLclasCd><cnttgMlsfcCd>03990000</cnttgMlsfcCd><acntbkAmt>34336982</acntbkAmt><acntbkQy>25586</acntbkQy><cntrbtrCo>2</cntrbtrCo><cntrCo>2</cntrCo></item><item><stdrYm>201702</stdrYm><areaCd>01</areaCd><unitySignguCd>01000000</unitySignguCd><rceptSpctrCd>S00101</rceptSpctrCd><foodYn>N</foodYn><cnttgLclasCd>04000000</cnttgLclasCd><cnttgMlsfcCd>04040000</cnttgMlsfcCd><acntbkAmt>10513284</acntbkAmt><acntbkQy>4706</acntbkQy><cntrbtrCo>1</cntrbtrCo><cntrCo>1</cntrCo></item><item><stdrYm>201702</stdrYm><areaCd>01</areaCd><unitySignguCd>01000000</unitySignguCd><rceptSpctrCd>S00101</rceptSpctrCd><foodYn>N</foodYn><cnttgLclasCd>04000000</cnttgLclasCd><cnttgMlsfcCd

In [140]:
result = xmltodict.parse(html)
dictionay = json.loads(json.dumps(result))["response"]["body"]["items"]["item"]
df_out = pd.DataFrame(dictionay)
df_out

,stdrYm,areaCd,unitySignguCd,rceptSpctrCd,foodYn,cnttgLclasCd,cnttgMlsfcCd,acntbkAmt,acntbkQy,cntrbtrCo,cntrCo
0,201702,01,01000000,S00101,N,03000000,03990000,34336982,25586,2,2
1,201702,01,01000000,S00101,N,04000000,04040000,10513284,4706,1,1
2,201702,01,01000000,S00101,N,04000000,04880000,78774,24,1,1
3,201702,01,01000000,S00101,Y,01000000,01010000,2545840,3464,1,24
4,201702,01,01000000,S00101,Y,01000000,01020000,11146771,18186,4,7
5,201702,01,01000000,S00101,Y,01000000,01030000,570000,100,1,1
6,201702,01,01000000,S00101,Y,01000000,01040000,32569848,3804,1,1
7,201702,01,01000000,S00101,Y,01000000,01050000,6354850,989,2,2
8,201702,01,01000000,S00101,Y,01000000,01100000,4103000,373,1,1
9,201702,01,01000000,S00101,Y,01000000,01150000,1353450,400,1,1


In [141]:

for col_ in df_out.columns:
    target_ = query_item[query_item["구분"]==col_]
    dic_tar = {col["코드"]:col["명"] for row,col in target_.iterrows()}
    col_list = []
    if len(target_)!=0:
        data_ = df_out[col_].map(lambda x: int(x))#.map(lambda x: x[1:] if x[0]=="0" else())
        data_2 = [dic_tar[i] for i in data_.values]
        df_out[col_] = data_2
    else:
        pass

In [142]:
df_out = df_out.rename(columns={col:cate_[col] for col in df_out.columns})
df_out

,기준년월,지역코드,통합시군구코드,접수지원센터코드,식품여부,기부물품대분류코드,기부물품중분류코드,장부금액,장부수량,기부자수,기부건수
0,201702,중앙,중앙,S00101,N,일상용품,기타일상용품,34336982,25586,2,2
1,201702,중앙,중앙,S00101,N,가정용품,생활용품,10513284,4706,1,1
2,201702,중앙,중앙,S00101,N,가정용품,가정용품세트류,78774,24,1,1
3,201702,중앙,중앙,S00101,Y,가공식품,조미료,2545840,3464,1,24
4,201702,중앙,중앙,S00101,Y,가공식품,유제품,11146771,18186,4,7
5,201702,중앙,중앙,S00101,Y,가공식품,축산가공식품,570000,100,1,1
6,201702,중앙,중앙,S00101,Y,가공식품,수산가공식품,32569848,3804,1,1
7,201702,중앙,중앙,S00101,Y,가공식품,대용식기타,6354850,989,2,2
8,201702,중앙,중앙,S00101,Y,가공식품,어육/연식품류,4103000,373,1,1
9,201702,중앙,중앙,S00101,Y,가공식품,건강식품,1353450,400,1,1
